In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import os
os.environ["KERAS_BACKEND"] = "tensorflow"  # "jax" or "tensorflow" or "torch" 

import keras_cv
import keras
import keras.backend as K
import tensorflow as tf
import tensorflow_io as tfio

import numpy as np 
import pandas as pd

from glob import glob
from tqdm import tqdm

import librosa
import IPython.display as ipd
import librosa.display as lid

import matplotlib.pyplot as plt
import matplotlib as mpl

cmap = mpl.cm.get_cmap('coolwarm')

2024-11-11 11:31:12.705243: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-11 11:31:12.705360: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-11 11:31:12.847739: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/tmp/ipykernel_34/1585041062.py:23: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap = mpl.cm.get_cmap('coolwarm')


In [3]:
print("TensorFlow:", tf.__version__)
print("Keras:", keras.__version__)
print("KerasCV:", keras_cv.__version__)

TensorFlow: 2.15.0
Keras: 3.3.3
KerasCV: 0.9.0


In [4]:
class CFG:
    seed = 42
    
    # Input image size and batch size
    img_size = [128, 384]
    batch_size = 64
    
    # Audio duration, sample rate, and length
    duration = 15 # second
    sample_rate = 32000
    audio_len = duration*sample_rate
    
    # STFT parameters
    nfft = 2028
    window = 2048
    hop_length = audio_len // (img_size[1] - 1)
    fmin = 20
    fmax = 16000
    
    # Number of epochs, model name
    epochs = 5
    preset = 'efficientnetv2_b2_imagenet'
    
    # Data augmentation parameters
    augment=True

    # Class Labels for BirdCLEF 24
    class_names = sorted(os.listdir('/kaggle/input/birdclef-2024/train_audio/'))
    num_classes = len(class_names)
    class_labels = list(range(num_classes))
    label2name = dict(zip(class_labels, class_names))
    name2label = {v:k for k,v in label2name.items()}

In [5]:
tf.keras.utils.set_random_seed(CFG.seed)

In [6]:
BASE_PATH = '/kaggle/input/birdclef-2024'


In [7]:

test_paths = glob(f'/kaggle/input/birdclef-2024/unlabeled_soundscapes/*ogg')

In [8]:
tf.keras.utils.set_random_seed(CFG.seed)


BASE_PATH = '/kaggle/input/birdclef-2024'


test_paths = glob(f'{BASE_PATH}/test_soundscapes/*ogg')
# During commit use `unlabeled` data as there is no `test` data.
# During submission `test` data will automatically be populated.
if len(test_paths)==0:
    test_paths = glob(f'{BASE_PATH}/unlabeled_soundscapes/*ogg')[:10]
test_df = pd.DataFrame(test_paths, columns=['filepath'])
test_df.head()

,filepath
0,/kaggle/input/birdclef-2024/unlabeled_soundsca...
1,/kaggle/input/birdclef-2024/unlabeled_soundsca...
2,/kaggle/input/birdclef-2024/unlabeled_soundsca...
3,/kaggle/input/birdclef-2024/unlabeled_soundsca...
4,/kaggle/input/birdclef-2024/unlabeled_soundsca...


In [9]:
test_paths

['/kaggle/input/birdclef-2024/unlabeled_soundscapes/1872382287.ogg',
 '/kaggle/input/birdclef-2024/unlabeled_soundscapes/161691311.ogg',
 '/kaggle/input/birdclef-2024/unlabeled_soundscapes/884633066.ogg',
 '/kaggle/input/birdclef-2024/unlabeled_soundscapes/1675425185.ogg',
 '/kaggle/input/birdclef-2024/unlabeled_soundscapes/1469579221.ogg',
 '/kaggle/input/birdclef-2024/unlabeled_soundscapes/1564606141.ogg',
 '/kaggle/input/birdclef-2024/unlabeled_soundscapes/194233408.ogg',
 '/kaggle/input/birdclef-2024/unlabeled_soundscapes/2003586554.ogg',
 '/kaggle/input/birdclef-2024/unlabeled_soundscapes/1459749597.ogg',
 '/kaggle/input/birdclef-2024/unlabeled_soundscapes/33854460.ogg']

In [10]:
# Create an input layer for the model
inp = keras.layers.Input(shape=(None, None, 3))
# Pretrained backbone
backbone = keras_cv.models.EfficientNetV2Backbone.from_preset(
    CFG.preset,
)
out = keras_cv.models.ImageClassifier(
    backbone=backbone,
    num_classes=CFG.num_classes,
    name="classifier"
)(inp)
# Build model
model = keras.models.Model(inputs=inp, outputs=out)


Attaching 'config.json' from model 'keras/efficientnetv2/keras/efficientnetv2_b2_imagenet/2' to your Kaggle notebook...
Attaching 'config.json' from model 'keras/efficientnetv2/keras/efficientnetv2_b2_imagenet/2' to your Kaggle notebook...
Attaching 'model.weights.h5' from model 'keras/efficientnetv2/keras/efficientnetv2_b2_imagenet/2' to your Kaggle notebook...


In [11]:
def check_file_existence(file_path):
    # Check if the file exists
    if os.path.exists(file_path):
        print(f"The file '{file_path}' exists.")
        return True
    else:
        print(f"The file '{file_path}' does not exist.")
        return False

# Example usage
file_path = '/kaggle/input/birdclef-24/other/fine_tune_efficient_net/1/best_model_90.weights.h5'
check_file_existence(file_path)

The file '/kaggle/input/birdclef-24/other/fine_tune_efficient_net/1/best_model_90.weights.h5' exists.


True

In [12]:
model.load_weights("/kaggle/input/birdclef-24/other/fine_tune_efficient_net/1/best_model_90.weights.h5")

In [13]:

# Decodes Audio
def build_decoder(with_labels=True, dim=1024):
    def get_audio(filepath):
        file_bytes = tf.io.read_file(filepath)
        audio = tfio.audio.decode_vorbis(file_bytes) # decode .ogg file
        audio = tf.cast(audio, tf.float32)
        if tf.shape(audio)[1]>1: # stereo -> mono
            audio = audio[...,0:1]
        audio = tf.squeeze(audio, axis=-1)
        return audio
    
    def create_frames(audio, duration=5, sr=32000):
        frame_size = int(duration * sr)
        audio = tf.pad(audio[..., None], [[0, tf.shape(audio)[0] % frame_size], [0, 0]]) # pad the end
        audio = tf.squeeze(audio) # remove extra dimension added for padding
        frames = tf.reshape(audio, [-1, frame_size]) # shape: [num_frames, frame_size]
        print(frames.shape)
        return frames
    
    def apply_preproc(spec):
        # Standardize
        mean = tf.math.reduce_mean(spec)
        std = tf.math.reduce_std(spec)
        spec = tf.where(tf.math.equal(std, 0), spec - mean, (spec - mean) / std)

        # Normalize using Min-Max
        min_val = tf.math.reduce_min(spec)
        max_val = tf.math.reduce_max(spec)
        spec = tf.where(tf.math.equal(max_val - min_val, 0), spec - min_val,
                              (spec - min_val) / (max_val - min_val))
        return spec

    def decode(path):
        # Load audio file
        audio = get_audio(path)
        # Split audio file into frames with each having 5 seecond duration
        audio = create_frames(audio)
        audio = apply_preproc(audio)
        # Convert audio to spectrogram
        spec = keras.layers.MelSpectrogram(num_mel_bins=CFG.img_size[0],
                                             fft_length=CFG.nfft, 
                                              sequence_stride=CFG.hop_length, 
                                              sampling_rate=CFG.sample_rate)(audio)
        # Apply normalization and standardization
        spec = apply_preproc(spec)
        # Covnert spectrogram to 3 channel image (for imagenet)
        spec = tf.tile(spec[..., None], [1,  1, 1, 3])
        print(spec.shape)

        return spec
     
    return decode


In [14]:
def build_dataset(paths, batch_size=1, decode_fn=None, cache=False):
    if decode_fn is None:
        decode_fn = build_decoder(dim=CFG.audio_len) # decoder
    AUTO = tf.data.experimental.AUTOTUNE
    slices = (paths,)
    ds = tf.data.Dataset.from_tensor_slices(slices)
    ds = ds.map(decode_fn, num_parallel_calls=AUTO) # decode audio to spectrograms then create frames
    ds = ds.cache() if cache else ds # cache files
    ds = ds.batch(batch_size, drop_remainder=False) # create batches
    ds = ds.prefetch(AUTO)
    return ds

In [15]:
import time

ids = []

# Initialize empty array to store predictions
preds = np.empty(shape=(0, CFG.num_classes), dtype='float32')

# Build test dataset
test_paths = test_df.filepath.tolist()
test_ds = build_dataset(paths=test_paths, batch_size=1)

files_handled = 0
start_time = time.time()

# Iterate over each audio file in the test dataset
for idx, specs in enumerate(tqdm(iter(test_ds), desc='test ', total=len(test_df))):
    # Extract the filename without the extension
    filename = test_paths[idx].split('/')[-1].replace('.ogg','')
    
    # Convert to backend-specific tensor while excluding extra dimension
    specs = keras.ops.convert_to_tensor(specs[0])
    
    # Predict bird species for all frames in a recording using all trained models
    frame_preds = model.predict(specs, verbose=0)
    
    # Create a ID for each frame in a recording using the filename and frame number
    frame_ids = [f'{filename}_{(frame_id+1)*5}' for frame_id in range(len(frame_preds))]
    
    # Concatenate the ids
    ids += frame_ids
    # Concatenate the predictions
    preds = np.concatenate([preds, frame_preds], axis=0)
    files_handled += 1

(None, 160000)
(None, 128, 128, 3)


test :   0%|          | 0/10 [00:00<?, ?it/s]WARNING: All log messages before absl::InitializeLog() is called are written to STDERR
I0000 00:00:1731324714.150494     103 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
test : 100%|██████████| 10/10 [00:28<00:00,  2.90s/it]


In [16]:
print(f"\n{files_handled} files processed in {(time.time() - start_time)} seconds")
avg_time = ((time.time() - start_time) / files_handled)
print(f"Time / file: {avg_time} seconds")
expected_files = 1100
total_hours = (avg_time * expected_files / 3600)
print(f"Estimated time for {expected_files}: {total_hours} hours")


10 files processed in 29.235992670059204 seconds
Time / file: 2.923610520362854 seconds
Estimated time for 1100: 0.8933254367775387 hours


In [17]:
pred_df = pd.DataFrame(ids, columns=['row_id'])
pred_df.loc[:, CFG.class_names] = preds
pred_df.to_csv('/kaggle/working/submission.csv',index=False)
pred_df.head()

,row_id,asbfly,ashdro1,ashpri1,ashwoo2,asikoe2,asiope1,aspfly1,aspswi1,barfly1,...,whbwoo2,whcbar1,whiter2,whrmun,whtkin2,woosan,wynlau1,yebbab1,yebbul3,zitcis1
0,1872382287_5,0.005227,0.002690,0.000633,0.000656,0.007912,0.000712,0.001363,0.000234,0.001599,...,0.007649,0.003922,0.001839,0.002102,0.004886,0.002420,0.000276,0.000181,0.000678,0.001340
1,1872382287_10,0.002365,0.003722,0.000372,0.000627,0.008708,0.000513,0.005525,0.000165,0.002143,...,0.003044,0.001162,0.005733,0.002083,0.003058,0.003102,0.000540,0.000233,0.000574,0.001559
2,1872382287_15,0.000854,0.015027,0.000441,0.001148,0.002472,0.000176,0.020864,0.000158,0.003749,...,0.001306,0.000825,0.005603,0.012217,0.001471,0.000646,0.001246,0.000306,0.000550,0.000745
3,1872382287_20,0.003600,0.002004,0.000477,0.000346,0.013661,0.001016,0.001575,0.000323,0.002536,...,0.003176,0.002458,0.004385,0.003759,0.002755,0.005720,0.000306,0.000406,0.000671,0.003012
4,1872382287_25,0.003129,0.000452,0.000473,0.000265,0.009421,0.000679,0.000728,0.000249,0.001138,...,0.001809,0.001507,0.004953,0.001737,0.001223,0.013611,0.000200,0.000296,0.000395,0.003765


In [18]:
pred_df

,row_id,asbfly,ashdro1,ashpri1,ashwoo2,asikoe2,asiope1,aspfly1,aspswi1,barfly1,...,whbwoo2,whcbar1,whiter2,whrmun,whtkin2,woosan,wynlau1,yebbab1,yebbul3,zitcis1
0,1872382287_5,0.005227,0.002690,0.000633,0.000656,0.007912,0.000712,0.001363,0.000234,0.001599,...,0.007649,0.003922,0.001839,0.002102,0.004886,0.002420,0.000276,0.000181,0.000678,0.001340
1,1872382287_10,0.002365,0.003722,0.000372,0.000627,0.008708,0.000513,0.005525,0.000165,0.002143,...,0.003044,0.001162,0.005733,0.002083,0.003058,0.003102,0.000540,0.000233,0.000574,0.001559
2,1872382287_15,0.000854,0.015027,0.000441,0.001148,0.002472,0.000176,0.020864,0.000158,0.003749,...,0.001306,0.000825,0.005603,0.012217,0.001471,0.000646,0.001246,0.000306,0.000550,0.000745
3,1872382287_20,0.003600,0.002004,0.000477,0.000346,0.013661,0.001016,0.001575,0.000323,0.002536,...,0.003176,0.002458,0.004385,0.003759,0.002755,0.005720,0.000306,0.000406,0.000671,0.003012
4,1872382287_25,0.003129,0.000452,0.000473,0.000265,0.009421,0.000679,0.000728,0.000249,0.001138,...,0.001809,0.001507,0.004953,0.001737,0.001223,0.013611,0.000200,0.000296,0.000395,0.003765
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
475,33854460_220,0.004145,0.002037,0.001667,0.001131,0.017875,0.000818,0.001224,0.005695,0.001042,...,0.004408,0.073656,0.004788,0.006899,0.003349,0.012494,0.000452,0.015104,0.000547,0.001968
476,33854460_225,0.004877,0.001318,0.001179,0.000816,0.008272,0.000514,0.000591,0.002486,0.000533,...,0.011763,0.210128,0.001771,0.003244,0.003298,0.002900,0.000355,0.004016,0.000707,0.001792
477,33854460_230,0.002422,0.003406,0.002142,0.001031,0.027336,0.000666,0.001494,0.005899,0.000683,...,0.009321,0.037942,0.003056,0.007153,0.005548,0.012552,0.000496,0.009856,0.000867,0.002272
478,33854460_235,0.002180,0.003785,0.001617,0.000688,0.034679,0.000545,0.000799,0.004359,0.000840,...,0.008000,0.033929,0.003315,0.003959,0.018962,0.022027,0.000374,0.008114,0.000796,0.001891
